In [1]:
import sqlite3

con = sqlite3.connect("/Users/upasana/sqlite/allsides-headline-roundup.db")
cur = con.cursor()

In [2]:
res = cur.execute("select tags from allsides_headlines where tags is not null")

In [3]:
lst = res.fetchall()

In [4]:
elt = lst[0]
split_list = elt[0].split(",")
cleaned_split_list = [x.strip() for x in split_list]
cleaned_split_list


['Gun Control And Gun Rights', 'Gun Violence', 'Police', 'Education']

In [5]:

dct: dict = {}

def clean_item(item: str | list):
    if type(item) == str:
        return item.replace(" ", "_").lower()
    else:
        return [x.replace(" ", "_").lower() for x in item]

for elt in lst:
    split_list = elt[0].split(",")
    cleaned_split_list = [x.strip() for x in split_list]
    for item in cleaned_split_list:
        item = clean_item(item)
        temp = clean_item(cleaned_split_list.copy())
        temp.remove(item)
        for neighbour in temp:
            neighbour = clean_item(neighbour)
            if item not in dct.keys():
                dct[item] = {}
            if neighbour not in dct[item].keys():
                dct[item][neighbour] = 0
            dct[item][neighbour] += 1


In [24]:
from utils import load_json

topics: list[str] = load_json("topics.json")['topics']

In [7]:
def top_k_keys(dct, k):
    # Sort the dictionary by value in descending order and get the top 5 keys
    sorted_items = sorted(dct.items(), key=lambda x: x[1], reverse=True)
    # Return the top k keys
    return [key for key, value in sorted_items[:min(len(dct) -1, k)]]

In [38]:
buckets: list[set] = []
topics_set = set(topics.copy())
bad_topics = []
import random

while topics_set:
    print(f"num topics left: {len(topics_set)}")
    selection = random.sample(topics_set, 1)[0]
    if selection in dct.keys():
        neighbours = set(top_k_keys(dct[selection], 5))
        neighbours.add(selection)
        neighbours = neighbours.intersection(topics_set)
        for bucket in buckets:
            intersect = bucket.intersection(neighbours)
            if intersect:
                leftover =  bucket.difference(neighbours)
                for l in leftover:
                    sims = [dct[l][x] for x in bucket if x in dct[l].keys()]
                    if len(sims) >= len(bucket) - (len(bucket) / 2):
                        bucket.add(l)
                        topics_set.remove(l)
                        neighbours.remove(l)

        if len(buckets) < 1 or len(neighbours) > 1:
            buckets.append(neighbours)
            topics_set = topics_set.difference(neighbours)
    else:
        print(f"key not in dct: {selection}")
        topics_set.remove(selection)
        bad_topics.append(selection)

    


num topics left: 108
num topics left: 104
num topics left: 100
num topics left: 95
key not in dct: bridging_divides
num topics left: 94
num topics left: 92
num topics left: 89
num topics left: 86
num topics left: 84
num topics left: 80
num topics left: 77
num topics left: 75
num topics left: 72
num topics left: 69
num topics left: 66
num topics left: 66
num topics left: 66
num topics left: 62
num topics left: 59
num topics left: 57
key not in dct: death_penalty
num topics left: 56
key not in dct: fiscal_cliff
num topics left: 55
num topics left: 55
num topics left: 55
num topics left: 53
num topics left: 53
num topics left: 51
num topics left: 51
num topics left: 51
num topics left: 51
num topics left: 51
num topics left: 48
num topics left: 48
num topics left: 46
num topics left: 46
num topics left: 46
num topics left: 46
key not in dct: homeland_security
num topics left: 45
num topics left: 45
num topics left: 45
num topics left: 45
num topics left: 45
num topics left: 43
key not in 

/var/folders/fz/y03qr7rx39zcncb4x6fsnclm0000gn/T/ipykernel_16709/3925452604.py:8: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  selection = random.sample(topics_set, 1)[0]


num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22
num topics left: 22


KeyboardInterrupt: 

In [34]:
bad_topics

['death_penalty',
 'homeland_security',
 'fiscal_cliff',
 'lgbt_rights',
 'dea',
 'bridging_divides',
 'tea_party']

In [39]:
print(f"number of buckets: {len(buckets)}")
print(f"sizes of buckets: {[len(x) for x in buckets]}")

number of buckets: 29
sizes of buckets: [4, 4, 5, 2, 3, 3, 2, 4, 3, 2, 3, 3, 3, 4, 3, 2, 2, 2, 3, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2]


In [40]:
buckets[26]

{'campaign_rhetoric', 'elections'}

In [62]:
import itertools
grouped_topics = load_json("gpt_topics.json")
grouped = list(itertools.chain(*grouped_topics.values()))

def find_duplicates(input_list):
    seen = set()
    duplicates = set()
    for item in input_list:
        if item in seen:
            duplicates.add(item)
        else:
            seen.add(item)
    return list(duplicates)

print(f"hallucinated topics: {set(grouped).difference(set(topics))}")
print(f"missing topics: {set(topics).difference(set(grouped))}")
print(f"num gpt topics: {len(set(grouped))}\num of total topics: {len(set(topics))}")
print(f"length of gpt topics list: {len(grouped)}\nlength of total topics list: {len(topics)}")
print(f"duplicates: {find_duplicates(grouped)}")


hallucinated topics: set()
missing topics: set()
num gpt topics: 108
um of total topics: 108
length of gpt topics list: 108
length of total topics list: 108
duplicates: []
